In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

file_path = 'data.csv'

data = pd.read_csv(file_path)


# Vérification des informations manquantes et des doublons
missing_data = data.isnull().sum()
duplicates = data.duplicated().sum()
data = data.drop_duplicates()


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


data.dropna(axis=0, inplace=True)


X = data.drop('charges', axis=1)
y = data['charges']


In [3]:
#Crée une colone de smoker en fonction du BMI
X['smoker_binary'] = (X['smoker'] == 'yes').astype(int)


#Création des intervalles pour les catégories BMI
bins = [0, 18.5, 24.9, 29.9, 34.9, 39.9, float('inf')]  # Les limites des catégories

#Étiquettes pour les catégories BMI
labels = [
    'underweight', 'normal weight', 'overweight',
    'obesity class I', 'obesity class II', 'obesity class III'
]

#Utilisation de pd.cut pour créer de nouvelles colonnes basées sur les catégories BMI
X['BMI_category'] = pd.cut(X['bmi'], bins=bins, labels=labels, right=False)

#Utilisation de pd.get_dummies pour obtenir des colonnes binaires pour chaque catégorie
BMI_dummies = pd.get_dummies(X['BMI_category'])

#Ajout des colonnes binaires au DataFrame X
X = pd.concat([X, BMI_dummies], axis=1)

X['bmi_smoker'] = X['bmi'] * X['smoker_binary']
X = X.drop('smoker_binary', axis=1)

#Suppression de la colonne 'BMI_category' car elle n'est plus nécessaire
X = X.drop('BMI_category', axis=1)

#Affichage du DataFrame avec les nouvelles colonnes binaires pour les catégories BMI
print(X)

      age     sex     bmi  children smoker     region  underweight  \
0      19  female  27.900         0    yes  southwest        False   
1      18    male  33.770         1     no  southeast        False   
2      28    male  33.000         3     no  southeast        False   
3      33    male  22.705         0     no  northwest        False   
4      32    male  28.880         0     no  northwest        False   
...   ...     ...     ...       ...    ...        ...          ...   
1333   50    male  30.970         3     no  northwest        False   
1334   18  female  31.920         0     no  northeast        False   
1335   18  female  36.850         0     no  southeast        False   
1336   21  female  25.800         0     no  southwest        False   
1337   61  female  29.070         0    yes  northwest        False   

      normal weight  overweight  obesity class I  obesity class II  \
0             False        True            False             False   
1             False

In [4]:

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])
# 80% pour train et 20% de test

print("Train set X", X_train.shape)
print("Train set Y", y_train.shape)
print("Test set X", X_test.shape)
print("Test set Y", y_test.shape)

Train set X (1136, 13)
Train set Y (1136,)
Test set X (201, 13)
Test set Y (201,)


In [5]:
# Identifier les colonnes catégories et numériques
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns


# Créer le pipeline pour les features numériques
numerical_pipeline = Pipeline([
    ('poly', PolynomialFeatures(2)), # Ajout de PolynomialFeatures
    ('scaler', StandardScaler())
    
])


# Créer le pipeline pour les features catégorielles
categorial_pipeline = Pipeline([
    ('encoder', OneHotEncoder()),
    ('poly', PolynomialFeatures(2))
])


# Combine les pipelines en utilisant ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, numerical_cols),
        ('categorial', categorial_pipeline, categorical_cols)
    ])


# Créer le pipeline final en ajoutant le model

LR_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regression', LinearRegression())
])

Lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('Lasso', Lasso())
])

ElasticNet_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('ElasticNet', ElasticNet(alpha=1e-07))
])


# On entraine les donnnées
LR_pipeline.fit(X_train, y_train)
Lasso_pipeline.fit(X_train, y_train)
ElasticNet_pipeline.fit(X_train, y_train)


/home/antoine/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.393e+09, tolerance: 1.667e+07
  model = cd_fast.enet_coordinate_descent(
/home/antoine/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.366e+10, tolerance: 1.667e+07
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('poly',
                                                                   PolynomialFeatures()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'bmi', 'children', 'bmi_smoker'], dtype='object')),
                                                 ('categorial',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder()),
                                                                  ('poly',
                                                                   PolynomialFeatures())]),
                                                  Index(['sex', 'smoker', 'region'], dtype='object'))])),
                ('ElasticNet', ElasticNet(alpha=1e-07))])

In [6]:
# On predicte avec LR
y_pred_LR = LR_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred_LR)
r2 = r2_score(y_test, y_pred_LR)
rmse = np.sqrt(mse)
print(f" score du LR modèle : {LR_pipeline.score(X_test, y_test)}")
print(f"mse : {mse}")
print(f"r2 : {r2}")
print(f"rmse : {rmse}")

 score du LR modèle : 0.9012578054089125
mse : 14309303.002365055
r2 : 0.9012578054089125
rmse : 3782.7639369071203


In [7]:
# On predicte avec Lasso
y_pred_Lasso = Lasso_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred_Lasso)
r2 = r2_score(y_test, y_pred_Lasso)
rmse = np.sqrt(mse)
print(f" score du Lasso modèle : {Lasso_pipeline.score(X_test, y_test)}")
print(f"mse : {mse}")
print(f"r2 : {r2}")
print(f"rmse : {rmse}")

 score du Lasso modèle : 0.9003179227707967
mse : 14445506.836107634
r2 : 0.9003179227707967
rmse : 3800.7245146297614


Recherche des meilleurs hyperparamètres de ElasticNet

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'ElasticNet__alpha': np.arange(1e-07),
    'ElasticNet__l1_ratio': np.arange(0.65, 0.90, 0.01)
}

grid = GridSearchCV(ElasticNet_pipeline, param_grid, cv= 5)

grid.fit(X_train, y_train)
grid.best_params_


/home/antoine/.local/lib/python3.10/site-packages/sklearn/base.py:1152: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/home/antoine/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/antoine/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.037e+10, tolerance: 1.315e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = c

{'ElasticNet__alpha': 0.0, 'ElasticNet__l1_ratio': 0.65}

Affichage du meilleur score :

In [9]:
model = grid.best_estimator_
model.score(X_test,y_test)

0.9003988116002952